In [1]:
import yaml
import csv
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
data_path = "../datasets/bikes_im/"

im_inds = []
im_names = []
im_prices = []
im_labels = []
with open('../datasets/bikes_classified.csv', 'r') as f:
        datareader = csv.reader(f)
        for line in datareader:
            im_inds.append(line[0])
            im_names.append(line[1])
            im_prices.append(line[2])
            im_labels.append(line[3])

data_cnn_features = np.load("../datasets/bikes_cnn_pca_features.npy")
print(data_cnn_features.shape)

(21843, 256)


In [3]:
# Use same train-test split as other models
train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

(19658,)
(2185,)


In [4]:
features_train = data_cnn_features[train_indices]
features_test = data_cnn_features[test_indices]
labels = np.array(im_labels)
labels_train = labels[train_indices]
labels_test = labels[test_indices]

In [5]:
classifier = svm.SVC(gamma=0.001)
classifier.fit(features_train, labels_train)

Wall time: 6min


In [8]:
predictions = classifier.predict(features_test)
from sklearn import metrics
print("Classification report:\n%s\n"
      % (metrics.classification_report(labels_test, predictions)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(labels_test, predictions))


Classification report:
             precision    recall  f1-score   support

        100       0.95      0.23      0.37       496
         25       0.98      0.27      0.43       596
         50       0.97      0.31      0.47       563
         75       0.30      0.98      0.46       530

avg / total       0.80      0.45      0.43      2185


Confusion matrix:
[[113   0   0 383]
 [  0 162   4 430]
 [  0   4 177 382]
 [  6   0   2 522]]


In [ ]:
# Hyperparameter tuning
# Note: this will take ~15 hours to complete and does not yield very fruitful results
'''
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma': gammas}

for score in ['precision', 'recall']:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=5, scoring='%s_macro' % score)
    search.fit(features_train, labels_train)
    print("Best parameters set found on development set:")
    print(search.best_params_)
    print("Grid scores on development set:")
    means = search.cv_results_['mean_test_score']
    stds = search.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, search.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    print("Detailed classification report:")
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    y_true, y_pred = labels_test, search.predict(features_test)
    print(classification_report(y_true, y_pred))
'''